# Midterm Assignment, topic 4: Equilibrium in the used cars market

> The sequence of tasks below should be performed in order in this
notebook. Feel free to add code and markdown cells, but do not change
cells that contain the assignment tasks.
If the task offers some degree of interpretation (so, don’t tell you what
to do exactly), you are free to interpret it in the way you see fit
best. But remember to explain why you make such and such choice in your answer.

> The grade for the midterm assignment is based on both the code
correctness and code style. The former relates to whether the code
adequately represents the economic model under consideration, and the
latter has to do with how well the code is modularized and organized,
whether appropriate syntax is used, and whether the code is well
documented. [PEP 8](https://www.python.org/dev/peps/pep-0008/)
standard is a good reference for coding style, but will not reduce the
grade.

## Equilibrium trade in the automobile market

In this exercise you will code up a dynamic model of equilibrium trade in the automobile market.

Look at the working paper “Equilibrium Trade In Automobile Markets” by
Kenneth Gillingham (Yale University), Fedor Iskhakov (ANU),
Anders Munk-Nielsen (University Of Copenhagen),
John Rust (Georgetown University) and Bertel Schjerning (University Of Copenhagen)
in the pdf file in the same repository.

The paper contains several gradually more complicated specifications of the
equilibrium model in sections 2, 3, 4 and 6.  Appendix A deals with the simplest
case of the stationary equilibrium with one car type and identical consumers.
It is this case that allows for an elegant algorithmic solution which is the point of
the assignment.

### Task 1. Model definition

Read section 2.1 for the general setup of the problem, and Appendix A for the
theory on the homogeneous consumer case.

Identify the fundamental parameters of the model:

- the upper bound on the age of the car $ A $  
- utility of car usage $ u(a) $  
- marginal utility of money $ \mu $  
- discount factor $ \beta $  
- accident probability function $ \alpha(a) $  
- maximum $ \overline{P} $ and minimum $ \underline{P} $ price on the market  


Identify the equilibrium variables:

- equilibrium price schedule $ P(a) $  
- equilibrium scrappage age  
- equilibrium holdings distribution  


Make sure you completely understand the meaning of all the variables entering
the Bellman equation 41-43:

- **State variables** — vector of variables that describe all relevant
  information about the modeled decision process  
- **Decision variables** — vector of variables describing the choices  
- **Instantaneous payoff** — utility function with time separable discounted utility  
- **Motion rules** — agent’s beliefs of how state variable evolve
  through time, conditional on choices  
- **Value function** — maximum attainable utility  
- **Policy function** — mapping from state space to action space that
  returns the optimal choice  

### Task 2. Design and implement a Python class to represent the model

- Let the attributes of the class to hold the fundamental scalar parameters
  of the model, and write the needed methods to return the accident probabilities.  
- Let the equilibrium objects also be represented by the attributes of the class  
- Implement *init* and *repr* methods.  
- Write a placeholder for the method that would compute the equilibrium of
  the model (a function with correct input and output arguments, which does
  not perform any operations).  You will write the functionality of this
  function in a later task.  
- Implement the method that computes stationary holding distribution conditional
  on the fundamental and the equilibrium objects (Hint: stationary holdings
  distribution is defined in A.D1, and physical transition matrix $ Q $ is
  given by equation 12)  
- Make a *plot* method for the class to illustrate the equilibrium price function
  as well as stationary holdings distribution (choose appropriate plot types).  

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import quantecon as qe

class Equilibrium_Trade():
    '''This class implements the equilibrium in automobile markets'''
    
    def __init__(self, 
                 u=lambda a: 60 - 5 * a,         # Utility function
                 α=lambda a: 0.01 + 0.02 * a,    # Accident probability function
                 β=0.95,                         # Discount Factor
                 μ=1.0,                      
                 σ=5, 
                 lower_p=1, 
                 upper_p=200,
                 A=20):
        
        # Store values
        self.u, self.α = u, α
        self.β, self.μ, self.σ = β, μ, σ
        self.lower_p, self.upper_p, self.A = lower_p, upper_p, A
    
    def X(self, γ):
        X_matrix = np.zeros((γ - 1, γ - 1))
        X_matrix[γ - 2, γ - 2] = self.α(γ - 2) * self.β - self.β - 1
        for i in range(γ - 2):
            X_matrix[i, i] = self.α(i) * self.β - self.β - 1
            X_matrix[i + 1, i] = 1.0
            X_matrix[i, i + 1] = self.β - self.α(i + 1) * self.β
        return X_matrix
    
    def Y(self, γ):
        Y_array = np.zeros(γ - 1)
        for i in range(γ - 1):
            if i == 0:
                Y_array[i] = (self.u(i) - self.u(i + 1))/self.μ + self.β * self.lower_p * (self.α(i) - self.α(i + 1)) - self.upper_p
            else:
                Y_array[i] = (self.u(i) - self.u(i + 1))/self.μ + self.β * self.lower_p * (self.α(i) - self.α(i + 1))
        return Y_array
    
    def Q(self, γ):
        matrix = np.zeros((γ, γ))
        row_γ = np.zeros((γ - 1, 1))
        for i in range(γ - 2):
            matrix[i, i + 1] = 1 - self.α(i)
            matrix[i, γ - 1] = self.α(i)
        matrix[γ - 1, 0] = 1 - self.α(0)
        matrix[γ - 2, γ - 1] = 1.0
        matrix[γ - 1, γ - 1] = self.α(0)
        return matrix
    
    def compute_equilibrium(self):
        for i in range(2, self.A + 1):
            P = np.linalg.solve(self.X(i), self.Y(i))
            P_next = np.linalg.solve(self.X(i + 1), self.Y(i + 1))
            P_bound =  np.linalg.solve(self.X(self.A), self.X(self.A))
            if (self.lower_p <= P).all() and (P_next <= self.upper_p).all():
                if (P_next < self.lower_p).any():
                    return i, P
            else:
                if (P_bound <= self.upper_p).all() and (self.lower_p <= P_bound).all():
                    return A, P_bound
    
    def holdings_distribution(self):
        γ, P = self.compute_equilibrium()
        q = self.Q(γ)
        mc = qe.MarkovChain(q)
        x = mc.stationary_distributions
        return x  
    
    def price_function(self):
        γ, P = self.compute_equilibrium()
        Price_Vector = np.ones(self.A)
        for i in range(P.size):
            Price_Vector[0] = self.upper_p
            if (i <= P.size):
                Price_Vector[i + 1] = P[i]
        return Price_Vector
    
    def plot(self):
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        γ, p = self.compute_equilibrium()
        price = self.price_function()
        q = self.holdings_distribution()
        x_grid = np.linspace(0, self.A, self.A)
        
        titles = ['(a) Equilibrium Prices', '(b) Post-Trade ownership distribution']
        
        for i in range(ax.size):
            ax[i].spines['right'].set_visible(False)
            ax[i].spines['top'].set_visible(False)
            ax[i].spines['left'].set_visible(True)
            ax[i].spines['bottom'].set_visible(True)
            ax[i].set_title(titles[i])
            ax[i].set_xlabel('Age of car')
        
        ax[0].set_xlim(0, γ)
        ax[0].set_ylim(0, price[0])
        ax[0].plot(x_grid, price)
        ax[1].bar(np.arange(len(q[0])), q[0], align='center')
        plt.tight_layout()
        plt.show()

ModuleNotFoundError: No module named 'quantecon'

In [ ]:
eq = Equilibrium_Trade()
a = eq.plot()

### Task 3. Plot the model for given values of parameters

Replicate the first two panels of Figure 1 from the paper using the same
parameters as mentioned in the text.
Use the parameter values listed in the paper.

### Task 4. Main step of the solution algorithm

Start writing the solver method of the class by coding up the linear system given in
the end of Appendix A in equation 58, and solving it using numpy linear solver.

This linear solver has to be called repeatedly for multiple values of the
scrappage age, therefore it could be written as a nested function inside of the
solver method.  Write this function with scrappage age as input, and let it return
the equilibrium prices.

Make a provision to be able to plot the computed equilibrium prices by the
object’s plot method.
Then make three plots of equilibrium prices for three scrappage ages of your choice.

### Task 5. Completed solution algorithm

Complete the solver method by writing the outer loop of the algorithm that would
find the optimal scrappage age.  Unlike suggested in the paper, use a simple iteration
over the possible scrap ages to find the one that satisfies the condition in Lemma A.L1.

### Task 6. Simple analysis

Compute and plot the stationary equilibrium prices and holdings distribution in the used
cars market for the following fundamentals:

- Let upper bound be $ A=50 $, price boundaries be $ (\underline{P},\overline{P})=(50,1) $,
  the discount factor be $ \beta=0.9 $, marginal utility of money equal $ \mu = 1.0 $,
  the utility function be $ u(a) = exp(4 - a/5) $, and the accident probability be
  $ \alpha(a) = 0.05 * Log(1 + x/3) $.  This is the baseline case.  
- Next, consider the static case where $ \beta=0 $.  What is the new equilibrium and
  why is it different  
- Return to the baseline model, and assume that $ u(a) = 10.0 $, i.e. the utility is
  independent of the age of the cars.  How does this equilibrium compare to the baseline? Why?  
- Again, return to the baseline model, and assume that the accident rate is twice higher for
  all ages of the cars.  Does this have an impact on the equilibrium outcome? Why?  
- Returning to the baseline model again, change the marginal utility of money to $ \mu = 2.0 $ replicating
  poorer consumers throughout the market.  How does this equilibrium compare to the baseline? Why?  